In [11]:
from src.vqe_runner import VQERunner
from src.ansatz_element_sets import UCCSDExcitations
from src.backends import MatrixCacheBackend, QiskitSimBackend
from src.cache import *
from src.molecules.molecules import H2, H4

import logging
import time

## Create a molecular system

In [12]:
# the unit for radius is an Angstrom
r = 1
molecule = H2(r=r)

## Create a UCCSD ansatz. 

In [13]:
n_orbitals = molecule.n_orbitals
n_electrons = molecule.n_electrons
ansatz_element_type = 'f_exc'
ansatz = UCCSDExcitations(n_orbitals, n_electrons, ansatz_element_type).get_all_elements()


## Choose a backend to calculate the expectation values for the energy.

Currently the code supports a qiskit noiseless simulator (QiskitSimBackend) and a custom matrix-multiplication noiseless simulator (MatrixCacheBackend).  

In [14]:
backend = QiskitSimBackend

# The MatrixCacheBackend uses a cache to save frequently computed values.
# The cache significantly accelarates the simulation speed, but it also
# requires a lot of RAM. Note that your computer will freeze if it runs out of RAM.
# You can safely use the cache for H2, but for any molecule with more than 
# 10 electrons it will require GBs of RAM

if backend == MatrixCacheBackend:
    global_cache = GlobalCache(molecule)
    global_cache.calculate_exc_gen_sparse_matrices_dict(ansatz)
else: 
    global_cache = None

## Create a VQE runner object

In [15]:
# the code supports all optimizers from scipy.minimize
optimizer = 'Nelder-Mead'
optimizer_options = {'gtol': 10e-8}
vqe_runner = VQERunner(molecule, backend=backend, use_ansatz_gradient=True,
                       optimizer=optimizer, optimizer_options=optimizer_options)

In [9]:
result = vqe_runner.vqe_run(ansatz=ansatz, cache=global_cache)

/home/jordan/anaconda3/envs/vqe_test/lib/python3.7/site-packages/scipy/optimize/_minimize.py:520: RuntimeWarning: Method Nelder-Mead does not use gradient information (jac).
  RuntimeWarning)
/home/jordan/workspace/VQE/src/vqe_runner.py:91: OptimizeWarning: Unknown solver options: gtol
  bounds=config.optimizer_bounds)


In [10]:
print(result.fun)

-1.1011503301329555
